# Data Import & Preprocessing

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
!pip install feature_engine
from feature_engine.creation import CyclicalFeatures
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
import numpy as np


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
fileName = "/content/sample_data/Dataset.xlsx"
df_Labels = pd.read_excel(fileName, sheet_name="Plant_1_Data")
df_Labels = df_Labels.drop(['PLANT_ID'], axis=1)
df_Labels["DATE_TIME"] = pd.to_datetime(df_Labels.DATE_TIME)
df_Features = pd.read_excel(fileName, sheet_name="Plant_1_Sensor_Data")
df_Features = df_Features.drop(['PLANT_ID','SOURCE_KEY'], axis=1)
df_Features['DATE_TIME'] = pd.to_datetime(df_Features.DATE_TIME)
print(df_Features.head())

            DATE_TIME  AMBIENT_TEMPERATURE  MODULE_TEMPERATURE  IRRADIATION
0 2020-05-15 00:00:00            25.184316           22.857507          0.0
1 2020-05-15 00:15:00            25.084589           22.761668          0.0
2 2020-05-15 00:30:00            24.935753           22.592306          0.0
3 2020-05-15 00:45:00            24.846130           22.360852          0.0
4 2020-05-15 01:00:00            24.621525           22.165423          0.0


In [24]:
df_Learn = pd.merge(df_Features, df_Labels, how='inner', left_on = ['DATE_TIME'], right_on = ['DATE_TIME'])
df_Learn.describe()

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
count,45680.000000,45680.000000,45680.000000,45680.000000,45680.000000,45680.000000,4.568000e+04
mean,25.917168,31.877975,0.236834,3197.175971,312.652679,3313.146538,6.957007e+06
std,3.556550,12.638448,0.306316,4080.448523,398.668968,3156.100252,4.172386e+05
min,20.398505,18.140415,0.000000,0.000000,0.000000,0.000000,6.183645e+06
25%,22.930031,21.406390,0.000000,0.000000,0.000000,0.000000,6.512358e+06
50%,24.993020,25.379072,0.035266,464.196429,44.912500,2653.633928,7.115711e+06
75%,28.379008,42.757119,0.459503,6478.424107,634.481250,6318.000000,7.244521e+06
max,35.252486,65.545714,1.221652,14471.125000,1410.950000,9163.000000,7.846821e+06


In [25]:
'''sourceKeyList = list(df_Learn['SOURCE_KEY'].unique())
sourceKeyDict = {}
for key in sourceKeyList:
  df_Key = df_Learn[df_Learn['SOURCE_KEY'] == key]
  #print(df_Key.describe())
  X = df_Key.drop(['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD', 'SOURCE_KEY'], axis = 1)
  #print(X.describe())
  y = df_Key['DC_POWER']
  
  #Train Val Test Split: 80%, 10%, 10%
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/9, random_state=42)
  #print(X_train.head()) print(y_train)
  sourceKeyDict[key] = list([X_train, y_train, X_val, y_val, X_test, y_test])
len(sourceKeyDict)
'''
X = df_Learn.drop(['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD', 'SOURCE_KEY'], axis = 1)
X["Hour"] =X["DATE_TIME"].dt.hour
X["Minute"]=X["DATE_TIME"].dt.minute
X=X.drop(["DATE_TIME"],axis=1)
y = np.log(df_Learn['DC_POWER'] + 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/9, random_state=42)
Scaler = StandardScaler()

column_to_scale =["AMBIENT_TEMPERATURE","MODULE_TEMPERATURE","IRRADIATION"]
Scaler.fit(X_train[column_to_scale])
X_train[column_to_scale] = Scaler.transform(X_train[column_to_scale])
X_test[column_to_scale] = Scaler.transform(X_test[column_to_scale])
display(X_train)

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,Hour,Minute
31161,-0.670111,-0.793333,-0.772531,1,45
43720,-0.883306,-0.822172,-0.772531,1,0
14303,1.518744,1.818756,1.887604,13,0
21089,-0.240173,-0.540894,-0.772531,21,30
20303,1.794998,2.120042,2.383081,12,30
...,...,...,...,...,...
11284,1.576698,1.931014,2.173641,13,15
44732,0.748897,0.922154,0.825447,13,0
38158,0.123579,0.902551,1.139332,10,0
860,0.763700,1.636051,1.416952,10,0


# Feature Manipulation

In [26]:

#df_Learn["Hour"] = df_Learn.DATE_TIME.dt.hour
#df_Learn["Minute"] = df_Learn.DATE_TIME.dt.minute
cyclical = CyclicalFeatures(variables=["Hour","Minute"], drop_original=True)
cyclical.fit(X_train)
X_train = cyclical.transform(X_train)
X_test = cyclical.transform(X_test)
display(X_train)

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,Hour_sin,Hour_cos,Minute_sin,Minute_cos
31161,-0.670111,-0.793333,-0.772531,0.269797,0.962917,-2.449294e-16,1.0
43720,-0.883306,-0.822172,-0.772531,0.269797,0.962917,0.000000e+00,1.0
14303,1.518744,1.818756,1.887604,-0.398401,-0.917211,0.000000e+00,1.0
21089,-0.240173,-0.540894,-0.772531,-0.519584,0.854419,-8.660254e-01,-0.5
20303,1.794998,2.120042,2.383081,-0.136167,-0.990686,-8.660254e-01,-0.5
...,...,...,...,...,...,...,...
11284,1.576698,1.931014,2.173641,-0.398401,-0.917211,8.660254e-01,-0.5
44732,0.748897,0.922154,0.825447,-0.398401,-0.917211,0.000000e+00,1.0
38158,0.123579,0.902551,1.139332,0.398401,-0.917211,0.000000e+00,1.0
860,0.763700,1.636051,1.416952,0.398401,-0.917211,0.000000e+00,1.0


# Model Training

In [27]:

'''pca_2 = PCA(n_components=2, random_state=42)
pca_2.fit(X_train)
x_pca_2 = pca_2.transform(X_train)
#pca_2.transform(X_test)
plt.figure(figsize=(10,7))
#sns.set(rc={'figure.figsize':(100,100)})'''
import plotly.express as px
from sklearn.decomposition import PCA

df = X_train
features = X_train.columns

pca = PCA()
components = pca.fit_transform(df[features])
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(2), width=800, height=600
)
fig.update_traces(diagonal_visible=False)
fig.show()
'''fig, ax = pyplot.subplots(figsize=(10,10))
sns.scatterplot(ax = ax, a=x_pca_2[:,0], b=x_pca_2[:, 1], s=70 , palette=['green','blue'])
plt.title("2D Scatterplot of dataset", pad = 15)
plt.xlabel("First principal component")
plt.ylabel("Second principal component")
plt.savefig("2d_scatterplot.png")
plt.plot(np.cumsum(pca_2.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Explained variance')
plt.savefig('Plot', dpi=100)'''

'fig, ax = pyplot.subplots(figsize=(10,10))\nsns.scatterplot(ax = ax, a=x_pca_2[:,0], b=x_pca_2[:, 1], s=70 , palette=[\'green\',\'blue\'])\nplt.title("2D Scatterplot of dataset", pad = 15)\nplt.xlabel("First principal component")\nplt.ylabel("Second principal component")\nplt.savefig("2d_scatterplot.png")\nplt.plot(np.cumsum(pca_2.explained_variance_ratio_))\nplt.xlabel(\'Number of components\')\nplt.ylabel(\'Explained variance\')\nplt.savefig(\'Plot\', dpi=100)'

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
parameter_range=parameter_range = np.arange(1, 10, 1)
SupportVectorRegModel = SVR(kernel='rbf', C = 8.0 , gamma = 0.2 , epsilon= 0.1)
#SupportVectorRegModel = SVR()
SupportVectorRegModel.fit(X_train,y_train)
display(cross_val_score(SupportVectorRegModel, X_train, y_train))
pred = SupportVectorRegModel.predict(X_test)
pred[pred<0]=0.0
display(pd.DataFrame(np.exp(pred)+1).describe())
display(pd.DataFrame(np.exp(y_test)+1).describe())
display(SupportVectorRegModel.score(X_test , y_test,sample_weight=None))



In [29]:
from scipy.sparse.linalg.isolve.minres import sqrt
from sklearn.metrics import mean_squared_error
mae = mean_absolute_error(y_test, pred)
display(mae)
display(np.exp(mae))
mse = mean_squared_error(y_test, pred)
display((mse))
rmse=sqrt(mse)
display((rmse))

0.31307761787237276

1.367627679258714

1.5910095214892974

0.68144606091157

In [30]:
vals = pd.DataFrame([[pd.to_datetime('2020-05-15 12:30:00'), '32.14768473',	'52.35325513',	'0.6492476293']], columns=('DATE_TIME', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION'))
#display(vals)
vals['Hour'] = vals['DATE_TIME'].dt.hour
vals['Minute'] = vals['DATE_TIME'].dt.minute
vals = vals.drop(['DATE_TIME'], axis = 1)
display(vals)
vals[column_to_scale] = Scaler.transform(vals[column_to_scale])
#display(vals)
vals = cyclical.transform(vals)
display(vals)
display("DC_POWER Prediction for sample: " + str(np.exp(SupportVectorRegModel.predict(vals))+1))

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,Hour,Minute
0,32.14768473,52.35325513,0.6492476293,12,30


,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,Hour_sin,Hour_cos,Minute_sin,Minute_cos
0,1.751636,1.620667,1.346999,-0.136167,-0.990686,-0.866025,-0.5


'DC_POWER Prediction for sample: [8801.384158]'

In [31]:
display("Size of samples: " + str(len(y_test)))
#Variance check for two tailed test
display("Variance of Predicted Samples: " + str(np.var(np.exp(pred)-1)), "Variance of Ground Truth: " + str(np.var(np.exp(y_test)-1)))
display("Standard Deviation of the Sample: " + str(np.exp(np.sqrt(np.var(pred)))))
#Ratio
display("Ratio to check for nearly equal Variance: " + str(np.var(y_test)/np.var(pred)))

'Size of samples: 4568'

'Variance of Predicted Samples: 16631326.082445342'

'Variance of Ground Truth: 16790437.655372243'

'Standard Deviation of the Sample: 59.46792686168183'

'Ratio to check for nearly equal Variance: 1.0590796772223754'

In [32]:
#T-Test
#Equal Variance, hence equal_var = 'True'
import scipy.stats as stats
tstat, t_pval = stats.ttest_ind(a=y_test, b=pred, equal_var=True)

display("T-Statistic: " + str(tstat.round(3)), "PValue: " + str(t_pval.round(3)))

'T-Statistic: 0.24'

'PValue: 0.81'